<a href="https://colab.research.google.com/github/cdiegor/Simulacao/blob/main/Filas_em_SimPy.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Filas M/M/· em SimPy — Caderno de Experimentos 🇧🇷

Este notebook reúne implementações em **SimPy** para:

- M/M/1  
- M/M/1/K (capacidade finita)  
- M/M/∞ (infinitos servidores)  
- M/M/c (c servidores)  
- M/M/1 com **prioridades** (2 classes, não-preemptiva)

e demonstra como obter, por simulação:

- tempos no sistema \\(W\\)
- tempos de espera \\(W_q\\)
- médias acumuladas para estudar **convergência para o regime estacionário**.

Use este caderno como base de aula e para propostas de laboratório.


In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import simpy

def make_rng(seed=None):
    """Cria um gerador NumPy com PCG64 (default_rng)."""
    return np.random.default_rng(seed)


## M/M/1 — 1 servidor, disciplina FIFO

Chegadas: processo de Poisson com taxa \\(\\lambda\\).  
Tempos de serviço: exponenciais com taxa \\(\\mu\\).  
Coletamos tempos no sistema \\(W\\) e tempos de espera \\(W_q\\).


In [ ]:
def simpy_mm1(lam, mu, T_max, seed=None):
    """
    Simulação de M/M/1 com chegadas Poisson(λ) e serviço Poisson(μ).

    Coleta tempos no sistema (W) e tempos de espera (Wq) para cada cliente
    atendido até o tempo T_max.
    """
    rng = make_rng(seed)
    env = simpy.Environment()
    server = simpy.Resource(env, capacity=1)

    W = []
    Wq = []

    def cliente(env, tempo_chegada):
        with server.request() as req:
            yield req
            inicio_servico = env.now
            espera = inicio_servico - tempo_chegada
            tempo_servico = rng.exponential(1 / mu)
            yield env.timeout(tempo_servico)
        permanencia = env.now - tempo_chegada
        Wq.append(espera)
        W.append(permanencia)

    def processo_chegadas(env):
        while True:
            inter = rng.exponential(1 / lam)
            yield env.timeout(inter)
            if env.now > T_max:
                break
            env.process(cliente(env, env.now))

    env.process(processo_chegadas(env))
    env.run(until=T_max)

    W = np.array(W)
    Wq = np.array(Wq)
    n = len(W)
    if n == 0:
        return {"W": np.array([]), "Wq": np.array([]),
                "W_med": np.array([]), "Wq_med": np.array([])}

    W_med = np.cumsum(W) / np.arange(1, n + 1)
    Wq_med = np.cumsum(Wq) / np.arange(1, n + 1)

    return {"W": W, "Wq": Wq, "W_med": W_med, "Wq_med": Wq_med}


## M/M/1/K — Capacidade finita \\(K\\)

Capacidade total \\(K\\) inclui o cliente em serviço.  
Se, na chegada, o sistema já tem \\(n_{\\text{sist}} \\ge K\\), o cliente é **bloqueado**.


In [ ]:
def simpy_mm1K(lam, mu, K, T_max, seed=None):
    """
    M/M/1/K com capacidade total K (inclui o servidor).
    Clientes que chegam com n_sistema >= K são bloqueados.
    """
    rng = make_rng(seed)
    env = simpy.Environment()
    server = simpy.Resource(env, capacity=1)

    n_sistema = 0
    chegados = 0
    bloqueados = 0

    W = []
    Wq = []

    def cliente(env, tempo_chegada):
        nonlocal n_sistema
        n_sistema += 1
        with server.request() as req:
            yield req
            inicio_servico = env.now
            espera = inicio_servico - tempo_chegada
            tempo_servico = rng.exponential(1 / mu)
            yield env.timeout(tempo_servico)
        permanencia = env.now - tempo_chegada
        Wq.append(espera)
        W.append(permanencia)
        n_sistema -= 1

    def processo_chegadas(env):
        nonlocal chegados, bloqueados
        while True:
            inter = rng.exponential(1 / lam)
            yield env.timeout(inter)
            if env.now > T_max:
                break
            chegados += 1
            if n_sistema >= K:
                bloqueados += 1
            else:
                env.process(cliente(env, env.now))

    env.process(processo_chegadas(env))
    env.run(until=T_max)

    W = np.array(W)
    Wq = np.array(Wq)
    n = len(W)
    if n == 0:
        return {"W": np.array([]), "Wq": np.array([]),
                "W_med": np.array([]), "Wq_med": np.array([]),
                "P_bloqueio": 0.0, "lambda_efetivo": 0.0,
                "chegados": chegados, "bloqueados": bloqueados}

    W_med = np.cumsum(W) / np.arange(1, n + 1)
    Wq_med = np.cumsum(Wq) / np.arange(1, n + 1)

    P_b = bloqueados / chegados if chegados > 0 else 0.0
    lam_eff = lam * (1 - P_b)

    return {"W": W, "Wq": Wq,
            "W_med": W_med, "Wq_med": Wq_med,
            "P_bloqueio": P_b, "lambda_efetivo": lam_eff,
            "chegados": chegados, "bloqueados": bloqueados}


## M/M/∞ — Infinitos servidores

Não há fila: cada cliente inicia serviço imediatamente.  
Logo, \\(W =\\) tempo de serviço \\(\\sim \\text{Exp}(\\mu)\\).


In [ ]:
def simpy_mminf(lam, mu, T_max, seed=None):
    """
    M/M/∞: cada cliente recebe servidor imediatamente, sem fila.
    Tempo no sistema = tempo de serviço ~ Exp(μ).
    """
    rng = make_rng(seed)
    env = simpy.Environment()

    W = []  # tempos no sistema (serviço)

    def cliente(env, tempo_chegada):
        tempo_servico = rng.exponential(1 / mu)
        yield env.timeout(tempo_servico)
        permanencia = env.now - tempo_chegada
        W.append(permanencia)

    def processo_chegadas(env):
        while True:
            inter = rng.exponential(1 / lam)
            yield env.timeout(inter)
            if env.now > T_max:
                break
            env.process(cliente(env, env.now))

    env.process(processo_chegadas(env))
    env.run(until=T_max)

    W = np.array(W)
    n = len(W)
    if n == 0:
        return {"W": np.array([]), "W_med": np.array([])}

    W_med = np.cumsum(W) / np.arange(1, n + 1)
    return {"W": W, "W_med": W_med}


## M/M/c — \\(c\\) servidores em paralelo

Chegadas Poisson(λ), serviços Exp(μ), disciplina FIFO, até \\(c\\) clientes em serviço simultaneamente.


In [ ]:
def simpy_mmc(lam, mu, c, T_max, seed=None):
    """
    M/M/c: c servidores idênticos, disciplina FIFO.
    """
    rng = make_rng(seed)
    env = simpy.Environment()
    server = simpy.Resource(env, capacity=c)

    W = []
    Wq = []

    def cliente(env, tempo_chegada):
        with server.request() as req:
            yield req
            inicio_servico = env.now
            espera = inicio_servico - tempo_chegada
            tempo_servico = rng.exponential(1 / mu)
            yield env.timeout(tempo_servico)
        permanencia = env.now - tempo_chegada
        Wq.append(espera)
        W.append(permanencia)

    def processo_chegadas(env):
        while True:
            inter = rng.exponential(1 / lam)
            yield env.timeout(inter)
            if env.now > T_max:
                break
            env.process(cliente(env, env.now))

    env.process(processo_chegadas(env))
    env.run(until=T_max)

    W = np.array(W)
    Wq = np.array(Wq)
    n = len(W)
    if n == 0:
        return {"W": np.array([]), "Wq": np.array([]),
                "W_med": np.array([]), "Wq_med": np.array([])}

    W_med = np.cumsum(W) / np.arange(1, n + 1)
    Wq_med = np.cumsum(Wq) / np.arange(1, n + 1)

    return {"W": W, "Wq": Wq, "W_med": W_med, "Wq_med": Wq_med}


## M/M/1 com prioridades (2 classes, não-preemptiva)

Duas classes de clientes:

- Classe 1 (alta prioridade): taxa \\(\\lambda_1\\)  
- Classe 2 (baixa prioridade): taxa \\(\\lambda_2\\)

Usamos `simpy.PriorityResource`: prioridade numérica menor = maior prioridade.


In [ ]:
def simpy_mm1_priority(lam1, lam2, mu, T_max, seed=None):
    """
    M/M/1 com duas classes de prioridade, NÃO-preemptiva.

    Classe 1: prioridade 0 (mais alta)
    Classe 2: prioridade 1 (mais baixa)
    """
    rng = make_rng(seed)
    env = simpy.Environment()
    server = simpy.PriorityResource(env, capacity=1)

    W1, Wq1 = [], []
    W2, Wq2 = [], []

    def cliente(env, tempo_chegada, classe, prioridade):
        with server.request(priority=prioridade) as req:
            yield req
            inicio_servico = env.now
            espera = inicio_servico - tempo_chegada
            tempo_servico = rng.exponential(1 / mu)
            yield env.timeout(tempo_servico)
        permanencia = env.now - tempo_chegada
        if classe == 1:
            Wq1.append(espera); W1.append(permanencia)
        else:
            Wq2.append(espera); W2.append(permanencia)

    def processo_chegadas(env, lam, classe, prioridade):
        while True:
            inter = rng.exponential(1 / lam)
            yield env.timeout(inter)
            if env.now > T_max:
                break
            env.process(cliente(env, env.now, classe, prioridade))

    env.process(processo_chegadas(env, lam1, classe=1, prioridade=0))
    env.process(processo_chegadas(env, lam2, classe=2, prioridade=1))
    env.run(until=T_max)

    def _stats(arr):
        arr = np.array(arr)
        if arr.size == 0:
            return arr, arr
        med = np.cumsum(arr) / np.arange(1, arr.size + 1)
        return arr, med

    W1, W1_med   = _stats(W1)
    Wq1, Wq1_med = _stats(Wq1)
    W2, W2_med   = _stats(W2)
    Wq2, Wq2_med = _stats(Wq2)

    return {
        "W1": W1, "Wq1": Wq1, "W1_med": W1_med, "Wq1_med": Wq1_med,
        "W2": W2, "Wq2": Wq2, "W2_med": W2_med, "Wq2_med": Wq2_med
    }


## Demonstrações rápidas de uso

Alguns cenários para gerar gráficos de convergência de \\(W_n\\) e comparar, em aula,
com os valores teóricos de \\(W, W_q\\) obtidos na lousa.


In [ ]:
# Exemplo: M/M/1 com ρ = 0.8 (λ=4, μ=5)
lam, mu = 4.0, 5.0
T_max = 2000.0

res = simpy_mm1(lam, mu, T_max=T_max, seed=1)
n = len(res["W_med"])
print(f"M/M/1: {n} clientes atendidos até T={T_max}")

plt.figure()
plt.plot(np.arange(1, n+1), res["W_med"], label="W médio (sim)")
plt.xlabel("nº de clientes")
plt.ylabel("W médio até o n-ésimo")
plt.title("M/M/1: convergência de W_n")
plt.legend()
plt.show()


In [ ]:
# Exemplo: M/M/1/K — estimando prob. de bloqueio
lam, mu, K = 4.0, 5.0, 10
T_max = 2000.0
resK = simpy_mm1K(lam, mu, K, T_max=T_max, seed=2)
print("M/M/1/K: chegados =", resK["chegados"],
      "bloqueados =", resK["bloqueados"],
      "P_bloqueio ≈", resK["P_bloqueio"])


In [ ]:
# Exemplo: M/M/1 com prioridades
lam1, lam2, mu = 2.0, 1.0, 5.0
T_max = 2000.0
resP = simpy_mm1_priority(lam1, lam2, mu, T_max=T_max, seed=3)

if resP["W1"].size:
    print("Classe 1 (alta): W médio ≈", resP["W1"].mean())
if resP["W2"].size:
    print("Classe 2 (baixa): W médio ≈", resP["W2"].mean())
